In [2]:
import os
import sys
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
%matplotlib inline
import seaborn as sns

sys.path.append('../')
from src.utils import parallel_apply

DIR = '/mnt/ml-team/minerva/open-solutions/home-credit'

In [3]:
description = pd.read_csv(os.path.join(DIR,'data/HomeCredit_columns_description.csv'),encoding = 'latin1')
application = pd.read_csv(os.path.join(DIR, 'files/unzipped_data/application_train.csv'))
installments = pd.read_csv(os.path.join(DIR, 'files/unzipped_data/installments_payments.csv'))

In [4]:
installments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


# Preprocessing
## Solution 3

# Feature Engineering
## Solution 3

## Aggregations

In [ ]:
INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES = []
for agg in ['mean', 'min', 'max', 'sum', 'var']:
    for select in ['AMT_INSTALMENT',
                   'AMT_PAYMENT',
                   'DAYS_ENTRY_PAYMENT',
                   'DAYS_INSTALMENT',
                   'NUM_INSTALMENT_NUMBER',
                   'NUM_INSTALMENT_VERSION'
                   ]:
        INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES.append((select, agg))
INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES = [(['SK_ID_CURR'], INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES)]

In [ ]:
groupby_aggregate_names = []
for groupby_cols, specs in tqdm(INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES):
    group_object = installments.groupby(groupby_cols)
    for select, agg in tqdm(specs):
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        application = application.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(index=str,
                                      columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
        groupby_aggregate_names.append(groupby_aggregate_name)

In [ ]:
application.head()

In [ ]:
application_agg = application[groupby_aggregate_names + ['TARGET']]
application_agg_corr = abs(application_agg.corr())

In [ ]:
application_agg_corr.sort_values('TARGET', ascending=False)['TARGET']

# Solution 4

In [ ]:
positive_ID = application[application['TARGET']==1]['SK_ID_CURR'].tolist()
positive_ID[:4]

In [ ]:
value_counts = installments[installments['SK_ID_CURR'].isin(positive_ID)]['SK_ID_CURR'].value_counts()

In [ ]:
value_counts.head()

In [ ]:
sns.distplot(value_counts)

In [ ]:
installments_one = installments[installments['SK_ID_CURR']==328162]

In [ ]:
installments_one.sort_values(['DAYS_INSTALMENT'],ascending=False).head(10)

In [5]:
installments_ = installments.sample(5000)
installments_['instalment_paid_late_in_days'] = installments_['DAYS_ENTRY_PAYMENT'] - installments_['DAYS_INSTALMENT'] 
installments_['instalment_paid_late'] = (installments_['instalment_paid_late_in_days'] > 0).astype(int)
installments_['instalment_paid_over_amount'] = installments_['AMT_PAYMENT'] - installments_['AMT_INSTALMENT']
installments_['instalment_paid_over'] = (installments_['instalment_paid_over_amount'] > 0).astype(int)

In [6]:
features = pd.DataFrame({'SK_ID_CURR':installments_['SK_ID_CURR'].unique()})
groupby = installments_.groupby(['SK_ID_CURR'])

In [7]:
def last_installment_features(gr):
    gr_ = gr.copy()
    gr_.sort_values(['DAYS_INSTALMENT'],ascending=False, inplace=True)
    last_instalment_id = gr_['SK_ID_PREV'].iloc[0]
    gr_ = gr_[gr_['SK_ID_PREV']==last_instalment_id]
    features = {}
    
    features = _add_agg_features(features,gr_, 'instalment_paid_late_in_days', 
                                 ['sum','mean','max','min','std'], 'last_')
    features = _add_agg_features(features,gr_ ,'instalment_paid_late', 
                                 ['count','mean'],'last_')
    features = _add_agg_features(features,gr_ ,'instalment_paid_over_amount', 
                                 ['sum','mean','max','min','std'],'last_')
    features = _add_agg_features(features,gr_,'instalment_paid_over', 
                                 ['count','mean'],'last_')

    return pd.Series(features)

def _add_agg_features(features, gr_, feature_name, aggs, prefix):
    for agg in aggs:
        if agg=='sum':
            features['{}{}_sum'.format(prefix, feature_name)] = gr_[feature_name].sum()
        elif agg =='mean':
            features['{}{}_mean'.format(prefix, feature_name)] = gr_[feature_name].mean()
        elif agg == 'max':
            features['{}{}_max'.format(prefix, feature_name)] = gr_[feature_name].max()
        elif agg == 'min':
            features['{}{}_min'.format(prefix, feature_name)] = gr_[feature_name].min()
        elif agg == 'std':
            features['{}{}_std'.format(prefix, feature_name)] = gr_[feature_name].std()
        elif agg =='count':
            features['{}{}_count'.format(prefix, feature_name)] = gr_[feature_name].count()
    return features

In [8]:
g = parallel_apply(groupby, last_installment_features, index_name='SK_ID_CURR',
                   num_workers=16, chunk_size=10000).reset_index()
features = features.merge(g, on='SK_ID_CURR', how='left')

display(features.head())

100%|██████████| 1/1.0 [00:05<00:00,  5.24s/it]


,SK_ID_CURR,last_instalment_paid_late_count,last_instalment_paid_late_in_days_max,last_instalment_paid_late_in_days_mean,last_instalment_paid_late_in_days_min,last_instalment_paid_late_in_days_std,last_instalment_paid_late_in_days_sum,last_instalment_paid_late_mean,last_instalment_paid_over_amount_max,last_instalment_paid_over_amount_mean,last_instalment_paid_over_amount_min,last_instalment_paid_over_amount_std,last_instalment_paid_over_amount_sum,last_instalment_paid_over_count,last_instalment_paid_over_mean
0,289198,1.0,-13.0,-13.0,-13.0,NaN,-13.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0
1,252830,1.0,-1.0,-1.0,-1.0,NaN,-1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0
2,448921,1.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0
3,395725,1.0,-8.0,-8.0,-8.0,NaN,-8.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0
4,137658,1.0,-9.0,-9.0,-9.0,NaN,-9.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0


In [4]:
def add_features(feature_name, aggs, features, feature_names, groupby):
    feature_names.extend(['{}_{}'.format(feature_name,agg) for agg in aggs])

    for agg in aggs:
        g = groupby[feature_name].agg(agg).reset_index().rename(index=str, 
                                                         columns={feature_name:'{}_{}'.format(feature_name,agg)})
        features = features.merge(g, on='SK_ID_CURR', how='left')
    return features, feature_names

In [5]:
feature_names = []

features, feature_names = add_features('instalment_paid_late_in_days', ['sum','mean','max','min','std'],
                                     features, feature_names, groupby)

features, feature_names = add_features('instalment_paid_late', ['sum','mean'],
                                     features, feature_names, groupby)

features, feature_names = add_features('instalment_paid_over_amount', ['sum','mean','max','min','std'],
                                     features, feature_names, groupby)

features, feature_names = add_features('instalment_paid_over', ['sum','mean'],
                                     features, feature_names, groupby)
    
display(features.head())

,SK_ID_CURR,instalment_paid_late_in_days_sum,instalment_paid_late_in_days_mean,instalment_paid_late_in_days_max,instalment_paid_late_in_days_min,instalment_paid_late_in_days_std,instalment_paid_late_sum,instalment_paid_late_mean,instalment_paid_over_amount_sum,instalment_paid_over_amount_mean,instalment_paid_over_amount_max,instalment_paid_over_amount_min,instalment_paid_over_amount_std,instalment_paid_over_sum,instalment_paid_over_mean
0,314475,0.0,0.0,0.0,0.0,NaN,0,0,0.0,0.0,0.0,0.0,NaN,0,0
1,208994,1.0,1.0,1.0,1.0,NaN,1,1,0.0,0.0,0.0,0.0,NaN,0,0
2,428307,-16.0,-16.0,-16.0,-16.0,NaN,0,0,0.0,0.0,0.0,0.0,NaN,0,0
3,349573,-6.0,-6.0,-6.0,-6.0,NaN,0,0,0.0,0.0,0.0,0.0,NaN,0,0
4,440440,-11.0,-11.0,-11.0,-11.0,NaN,0,0,0.0,0.0,0.0,0.0,NaN,0,0


In [ ]:
X = application.merge(features, on='SK_ID_CURR',how='left')
X = X[features.columns.drop('SK_ID_CURR').tolist()+['TARGET']]
X.head()

In [ ]:
X_corr = abs(X.corr())
X_corr.sort_values('TARGET', ascending=False)['TARGET']